# Using the Quantum Approximate Optimization Algorithm (QAOA)

The [QAOA](https://arxiv.org/abs/1411.4028) is a near-term quantum algorithm for approximately solving optimization problems. This notebook will walk you through how to use Qiskit Aqua to solve MaxCut and other combinatorial optimization problems with the QAOA.

## Learning goals

(1) Understand the goal of MaxCut.

(2) Know how to map an optimization problem (e.g., MaxCut) to a QAOA problem using Qiskit.

(3) Be able to find the optimal parameters for the circuit using optimizers in Qiskit.

(4) Understand how to sample from the QAOA circuit with optimal parameters to obtain approximate solutions.

In [ ]:
"""Imports for the notebook."""
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

import qiskit
print(qiskit.__qiskit_version__)

In [ ]:
"""Optional: Turn off warnings for the notebook."""
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
"""Specific imports for QAOA with MaxCut."""
# Import the QAOA object
from qiskit.aqua.algorithms.adaptive import QAOA

# Import tools for the MaxCut problem
from qiskit.aqua.translators.ising.max_cut import random_graph, get_max_cut_qubitops
from qiskit.aqua.operators.weighted_pauli_operator import Pauli, WeightedPauliOperator

# Import optimizers in Qiskit for finding the best parameters in the QAOA circuit
from qiskit.aqua.components.optimizers import ADAM, AQGD, COBYLA, POWELL

In [ ]:
"""Helper functions and other useful code."""
def draw_weighted(graph: nx.Graph,
                  pos_color: str = "blue",
                  neg_color: str = "red",
                  scale: float = 2.0) -> None:
    """Shows a visual of a graph with edges scaled by weight and colored by sign.
    
    Args:
        graph: The weighted graph to visualize.
        scale: Floating point value to scale edge weights by
               in the visualization. Purely aesthetic.
    
    """
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph, pos, node_size=700)
    
    col = lambda sgn: pos_color if sgn > 0 else neg_color
    
    for edge in graph.edges:
        weight = graph.get_edge_data(*edge)["weight"]
        sgn = np.sign(weight)
        size = abs(weight)
        nx.draw_networkx_edges(graph, 
                               pos, 
                               edgelist=[edge], 
                               width=scale * size,
                               edge_color=col(sgn),
                               alpha=0.5)
    nx.draw_networkx_labels(graph, pos, font_size=20)
    plt.axis("off")
    plt.show()

# Using the QAOA from Qiskit Aqua

## Step 1: Define the problem

In [ ]:
"""Define the graph for MaxCut via an adjacency matrix."""
matrix = random_graph(6, edge_prob=0.5)
print("The adjacency matrix is:")
print(matrix)

In [ ]:
"""Convert the adjacency matrix to a (weighted) graph and visualize it."""
graph = nx.from_numpy_array(matrix, parallel_edges=False)
draw_weighted(graph, pos_color="red", neg_color="blue")

### Try a cut!

## Step 2: Translate the problem to quantum

In this step, we input an adjacency matrix describing a weighted graph and output a string of Pauli operators and constant shift which defines the MaxCut cost function.

In [ ]:
"""Pauli operators from matrix."""
op, shift = get_max_cut_qubitops(matrix)

In [ ]:
"""Inspect the Pauli operators."""
print(f"There are {len(op.paulis)} weighted Pauli operators.")
for pauli in op.paulis:
    print(pauli[0], "*", pauli[1].to_label())

## Step 3: Define the QAOA circuit(s)

In [ ]:
"""Make the QAOA instance."""
qaoa = QAOA(op, COBYLA(), p=1)
print(qaoa.print_settings())

In [ ]:
"""Inspect the circuits."""
backend = qiskit.BasicAer.get_backend("qasm_simulator")
circs = qaoa.construct_circuit([1, 2], backend=backend)

print(f"There are {len(circs)} circuits.")
print(circs[0])

## Step 4: Sweep the leg (parameters)

In [ ]:
N = 15
gammas = np.linspace(-np.pi, np.pi, N)
betas = np.linspace(-np.pi, np.pi, N)

In [ ]:
quantum_instance = qiskit.aqua.QuantumInstance(backend=qiskit.BasicAer.get_backend("qasm_simulator"))
qaoa._quantum_instance = quantum_instance
qaoa._use_simulator_operator_mode = True

In [ ]:
import progressbar
bar = progressbar.ProgressBar(maxval=N**2)

costs = np.zeros((len(gammas), len(betas)), dtype=float)
bar.start()
for (ii, gamma) in enumerate(gammas):
    for (jj, beta) in enumerate(betas):
        costs[ii][jj] = qaoa._energy_evaluation(np.array([gamma, beta]))
        bar.update(N * ii + jj)
bar.finish()

In [ ]:
"""Visualize the landscape."""
plt.figure(figsize=(7, 7));
plt.imshow(costs, origin=(0, 0));
plt.xlabel("Gammas")
plt.ylabel("Betas")
plt.colorbar();

## Step 5: Run the optimizer

Here, we run the optimizer to get the best angles -- i.e., the angles which produce the lowest cost value.

In [ ]:
"""Get a quantum instance and run the algorithm."""
qaoa._optimizer = POWELL()
result = qaoa.run(quantum_instance)

## Step 6: Parse the output

In [ ]:
print(result)

In [ ]:
qaoa.get_optimal_cost()

In [ ]:
print(qaoa.print_settings())

## Step 7: Sample from the circuit with optimal parameters

In [ ]:
"""Get the circuit with optimal parameters."""
circ = qaoa.get_optimal_circuit()
qreg = circ.qregs[0]
creg = qiskit.ClassicalRegister(6)
circ.add_register(creg)
circ.measure(qreg, creg)
print(circ)

In [ ]:
"""Execute the circuit to sample from it."""
job = qiskit.execute(circ, backend=backend, shots=1000)
res = job.result()
counts = res.get_counts()

import operator
sorted_counts = sorted(counts.items(), key=operator.itemgetter(1))
print(sorted_counts)

In [ ]:
qiskit.visualization.plot_histogram(counts, figsize=(17, 8))

#### Do your top sampled bit strings provide a good cut for the graph?

# Questions and exercises

## How do different optimizers compare?

We used the `COBYLA` optimizer above, but there are others in `aqua.components.optimizers`. Pick a few of these and compare their performance. (How will you compare performance?)

## Is this the optimal cost?

Can you get a lower cost by increasing the number of layers (the `p` parameter in QAOA)?

How does this answer compare to classical solutions?

## Try another problem!

We used QAOA for the MaxCut problem above, but you can use it for many other combinatorial optimization problems. Pick one (in Qiskit Aqua) and do so!